In [1]:
from rdflib import Graph, Literal, Namespace
from rdflib.namespace import RDFS
import hashlib

g = Graph()

terms = Namespace("http://related-terms.io/terms/")
relations = Namespace("http://related-terms.io/relations/")
termsv = Namespace("http://related-terms.io/vocab#")

relaredToPred = termsv.relatedTo
weightPred = termsv.weight
labelPred = RDFS.label

def md5_hash(t):
    result = hashlib.md5(t.encode())
    return result.hexdigest()

def add_terms(t1, t2, w):
    t1 = t1.lower()
    t2 = t2.lower()
    t1h = md5_hash(t1)
    t2h = md5_hash(t2)
    g.add((terms[t1h], relaredToPred, terms[t2h]))
    g.add((terms[t2h], relaredToPred, terms[t1h]))
    g.add((terms[t1h], labelPred, Literal(t1)))
    g.add((terms[t2h], labelPred, Literal(t2)))
    g.add((relations[f"{t2h}-{t1h}"], weightPred, Literal(w)))
    g.add((relations[f"{t1h}-{t2h}"], weightPred, Literal(w)))


In [ ]:
from tqdm import tqdm
with open('data/merged-terms.csv') as f:
    for l in tqdm(f, total=3127145):
        l = l.strip().lower().split('|||')
        add_terms(l[0], l[1], float(l[2]))
            
g.serialize(destination="related-terms.nt", format='ntriples')

 48%|██████████████████████████████████████████████████████████████████████████▍                                                                                | 1502340/3127145 [05:42<08:42, 3111.97it/s]

In [3]:
!aws s3 cp related-terms.nt s3://osedemah-mesh-rdf/related-terms.nt

upload: ./related-terms.nt to s3://osedemah-mesh-rdf/related-terms.nt


In [4]:
!curl -X POST \
    -H 'Content-Type: application/json' \
    https://database-1.cluster-cclypxvaeodz.us-west-2.neptune.amazonaws.com:8182/loader -d '\
    {\
      "source" : "s3://osedemah-mesh-rdf/related-terms.nt",\
      "format" : "ntriples",\
      "iamRoleArn" : "arn:aws:iam::812630329048:role/NeptuneS3",\
      "region" : "us-west-2",\
      "failOnError" : "FALSE",\
      "parallelism" : "MEDIUM",\
      "updateSingleCardinalityProperties" : "FALSE",\
      "queueRequest" : "TRUE"\
    }'

{
    "status" : "200 OK",
    "payload" : {
        "loadId" : "cf5b9788-696b-40fb-94a3-5a37dc8cf23a"
    }
}

In [1]:
!curl https://database-1.cluster-cclypxvaeodz.us-west-2.neptune.amazonaws.com:8182/loader/cf5b9788-696b-40fb-94a3-5a37dc8cf23a

curl: (28) Failed to connect to database-1.cluster-cclypxvaeodz.us-west-2.neptune.amazonaws.com port 8182 after 75130 ms: Operation timed out
